# 吉珠自动健康卡填报

## 用到的库

不必了解每个库的具体用法，在下面的示范中尝试理解即可，有不懂的再去对应的文档里面查阅

### requests
### selenium

用于模拟浏览器发送和接受请求

官方文档：https://requests.readthedocs.io/en/master/user/quickstart/

如何安装：
>pip install requests

>pip install selenium

### json

用于读取和创建json文件

官方文档：https://docs.python.org/zh-cn/3/library/json.html

如何安装：内置已有


In [1]:
import requests
import json

## 关于身份验证

只需要一个人的登陆 Cookie 就可以提交和查看任意人的健康卡

如果 JSESSIONID 失效了需要手动在网站登陆一遍然后从Chrome开发者工具里找到：

![image](img/WX20200808-140955@2x.png)

**复制替换下方的 JSESSIONID 值即可。**

In [2]:
JSESSIONID = "DF74C430B19B50BA220ECF250598EEF9"
headers = {
    "x-requested-with": "XMLHttpRequest",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) ,Chrome/84.0.4147.105 Safari/537.36",
    "content-type": "text/json",
    "origin": "https://work.jluzh.edu.cn",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "cors",
    "sec-fetch-dest": "empty",
    "referer": "https://work.jluzh.edu.cn/default/work/jlzh/jkxxtb/jkxxcj.jsp",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-length": "896",
    "cookie": "JSESSIONID=" + JSESSIONID
}

In [3]:
# 写文件的辅助函数
def output_to_file(filename, data):
    with open(filename,'w') as fd:
        fd.write(json.dumps(data,indent=4,ensure_ascii=False))

## 查询填写记录

**查询当天填写记录(only_today=Ture) ** 可以用于判断当天有没有提交，辅导员据此看谁没有填健康卡

**查询历史提交记录(only_today=False) ** 顾名思义可以看到所有的提交记录

In [4]:
def query_record(number,only_today=True):
    # number = 20190105  # 学号
    # is_today = False   # 是否只查询当天

    if only_today:
        querySqlId = "com.sudytech.work.jlzh.jkxxtb.jkxxcj.queryToday"
    else:
        querySqlId = "com.sudytech.work.jlzh.jkxxtb.jkxxcj.queryNear"

    url = "https://work.jluzh.edu.cn/default/work/jlzh/jkxxtb/com.sudytech.portalone.base.db.queryBySqlWithoutPagecond.biz.ext"
    payloads = '{"params":{"empcode":"'+str(number)+'"},"querySqlId":"'+querySqlId+'"}'
    r = requests.post(url,headers = headers,data=payloads)

    try:
        output_to_file(f"data/{number}_temp_output.json",r.json())
        print(f"已将返回数据输出至 data/{number}_temp_output.json")
    except Exception as e:
        print(e)
        print(r.text)

    return r.json()

In [8]:
data  = query_record(20190226, False) # 查询

已将返回数据输出至 data/20190226_temp_output.json


## 提交

多次提交会在当日提交里创建多个记录，据网站说明会以最后一个提交为准

In [5]:
def submit(payloads):
    url = "https://work.jluzh.edu.cn/default/work/jlzh/jkxxtb/com.sudytech.portalone.base.db.saveOrUpdate.biz.ext"
    r = requests.post(url,headers = headers,data=payloads.encode("utf-8"))
    
    output_to_file("data/submit_temp_output.json",r.json())
    print("提交成功，返回数据已输出至 data/submit_temp_output.json")

本示例中直接读取本地的健康表填报文件，并作为结果发送。

运行前需确保文件中的时间以及其它个人信息是正确的

提交内容中的 **sqrid** 每个人都是不一样的，可以通过上面查找之前的填写记录获取 

In [66]:
with open("data/20190105_submit_demo.json",'r') as fd:
    data = json.load(fd)
# submit(json.dumps(data,ensure_ascii=False))

## 下一步要做什么

在给好Cookie的前提下，提供指定的学号，从提交历史记录中查询必要的信息构建用于本次提交的健康卡，然后提交该健康卡，并检验是否已经提交成功。

写一个可以运行的Python程序实现以上功能

## 尝试自动构造完整的健康卡

In [6]:
import time
last_card = query_record(20190105,False)["list"][0]
# last_card

已将返回数据输出至 data/20190105_temp_output.json


In [9]:
health_card_data = {
    "entity": {
        "sqrid": last_card["SQRID"],
        "sqbmid": last_card["SQBMID"],
        "fdygh": last_card["FDYGH"],
        "rysf": last_card["RYSF"],
        "bt": time.strftime("%Y-%m-%d", time.localtime()) + last_card["BT"][10:],
        "sqrmc": last_card["SQRMC"],
        "gh": last_card["GH"],
        "xb": last_card["XB"],
        "sqbmmc": last_card["SQBMMC"],
        "nj": last_card["NJ"],
        "zymc": last_card["ZYMC"],
        "bjmc": last_card["BJMC"],
        "fdymc": last_card["FDYMC"],
        "ssh": last_card["SSH"],
        "lxdh": last_card["LXDH"],
        "tbrq": time.strftime("%Y-%m-%d", time.localtime()),
        "tjsj": time.strftime("%Y-%m-%d %H:%M", time.localtime()) ,
        "xjzdz": last_card["XJZDZ"],
        "jqqx": last_card["JQQX"],
        "sfqwhb": last_card["SFQWHB"],
        "sfjchbjry": last_card["SFJCHBJRY"],
        "sfjwhy": last_card["SFJWHY"],
        "sfjwhygjdq": last_card["SFJWHYGJDQ"],
        "xrywz": last_card["XRYWZ"],
        "jtdz": last_card["JTDZ"],
        "grjkzk": last_card["GRJKZK"],
        "jrtw": last_card["JRTW"],
        "qsjkzk": last_card["QSJKZK"],
        "jkqk": last_card["JKQK"],
        "cn": [
            "本人承诺登记后、到校前不再前往其他地区"
        ],
        "bz": last_card["BZ"],
        "_ext": "{}",
        "__type": "sdo:com.sudytech.work.jlzh.jkxxtb.jkxxcj.TJlzhJkxxtb"
    }
}
# health_card_data

In [10]:
submit(json.dumps(health_card_data,ensure_ascii=False))

提交成功，返回数据已输出至 data/submit_temp_output.json


## 提交多天的健康卡

In [51]:
def muti_day_submit(days, addr):
    for i in range(days):
        last_card['JTDZ'] = addr
        last_card['XJZDZ'] = addr
        modtime = list(time.localtime())
        modtime[2] += i  
        modtime = time.struct_time(modtime)
        health_card_data = {
            "entity": {
                "sqrid": last_card["SQRID"],
                "sqbmid": last_card["SQBMID"],
                "fdygh": last_card["FDYGH"],
                "rysf": last_card["RYSF"],
                "bt": time.strftime("%Y-%m-%d", modtime) + last_card["BT"][10:],
                "sqrmc": last_card["SQRMC"],
                "gh": last_card["GH"],
                "xb": last_card["XB"],
                "sqbmmc": last_card["SQBMMC"],
                "nj": last_card["NJ"],
                "zymc": last_card["ZYMC"],
                "bjmc": last_card["BJMC"],
                "fdymc": last_card["FDYMC"],
                "ssh": last_card["SSH"],
                "lxdh": last_card["LXDH"],
                "tbrq": time.strftime("%Y-%m-%d", modtime),
                "tjsj": time.strftime("%Y-%m-%d %H:%M", modtime) ,
                "xjzdz": last_card["XJZDZ"],
                "jqqx": last_card["JQQX"],
                "sfqwhb": last_card["SFQWHB"],
                "sfjchbjry": last_card["SFJCHBJRY"],
                "sfjwhy": last_card["SFJWHY"],
                "sfjwhygjdq": last_card["SFJWHYGJDQ"],
                "xrywz": last_card["XRYWZ"],
                "jtdz": last_card["JTDZ"],
                "grjkzk": last_card["GRJKZK"],
                "jrtw": last_card["JRTW"],
                "qsjkzk": last_card["QSJKZK"],
                "jkqk": last_card["JKQK"],
                "cn": [
                    "本人承诺登记后、到校前不再前往其他地区"
                ],
                "bz": last_card["BZ"],
                "_ext": "{}",
                "__type": "sdo:com.sudytech.work.jlzh.jkxxtb.jkxxcj.TJlzhJkxxtb"
            }
        }
        submit(json.dumps(health_card_data,ensure_ascii=False))
        print(f'已提交 {time.strftime("%Y-%m-%d", modtime)} 健康卡')
muti_day_submit(9, "吉林大学珠海学院竹园二十栋101")

提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-12 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-13 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-14 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-15 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-16 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-17 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-18 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-19 健康卡
提交成功，返回数据已输出至 data/submit_temp_output.json
已提交 2021-01-20 健康卡


## 自动获取cookie

在微信小程序中填写健康卡抓包获取到登陆网站为

>https://wxapp.jluzh.com/cas/?target=https://work.jluzh.com/default/work/jlzh/jkxxtb/jkxxcj.jsp

![image](img/charles.jpg)

In [10]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from setting import * #账号密码从setting.py中读取

In [11]:
def getWeb():
    '''配置web浏览器'''
    options = webdriver.ChromeOptions()
    mobile_emulation = {"deviceName": "Galaxy S5"}
    capabilities = DesiredCapabilities.CHROME
    capabilities['loggingPrefs'] = {'browser': 'ALL'}
    # options = webdriver.ChromeOptions()
    options.add_experimental_option("mobileEmulation", mobile_emulation)
    options.add_experimental_option("excludeSwitches", ['enable-automation'])
    # 修改windows.navigator.webdriver，防机器人识别机制，selenium自动登陆判别机制desired_capabilities=capabilities,
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    # 更换头部
    options.add_argument('user-agent=mozilla/5.0 (iphone; cpu iphone os 5_1_1 like mac os x) applewebkit/534.46 (khtml, like gecko) mobile/9b206 micromessenger/5.0')
    #禁用图片
    #禁止图片和css加载
    prefs = {'permissions.default.stylesheet':2}#"profile.managed_default_content_settings.images": 2,
    options.add_experimental_option("prefs", prefs)
    # #添加代理
    # ip,port = '127.0.0.1','8080'
    # options.add_argument(('--proxy-server=http://{}:{}'.format(ip,port)))#有的博客写的是'--proxy-server=http://'，就目前我的电脑来看的话需要把http://去掉就可以用，他会自己加的
    # options.add_argument('-headless')  # 无头参数
    # options.headless = True
    web = webdriver.Chrome(executable_path=r'E:\Google\Chrome\Application\chromedriver.exe',options=options)
    url = "https://wxapp.jluzh.edu.cn/cas/?target=https://work.jluzh.com/default/work/jlzh/jkxxtb/jkxxcj.jsp"
    web.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
        })
    """
    })
    web.set_page_load_timeout(30)
    web.set_script_timeout(30)#这两种设置都进行才有效
    web.get(url)
    # web.add_cookie({'name': 'JSESSIONID','path': '/','value': cookies})
    return web

In [12]:
def get_cookie():
    web = getWeb()
    user = web.find_element_by_id("username")
    user.send_keys("{}".format(username))#账号密码从setting.py中读取
    passwd = web.find_element_by_id("password")
    passwd.send_keys("{}".format(password))#账号密码从setting.py中读取
    btn = web.find_element_by_id("passbutton")
    btn.click()
    cookie = web.get_cookies()[0]['value']
    web.quit()
    return cookie

## 部署服务器运行

1. 在centos服务器安装chrome浏览器

2. 下载chromedriver

3. 设定时间自动运行